In [0]:
class Queue():
    def __init__(self,initial):
        self.items = [initial]
    def isEmpty(self):
        return self.items == []
    def enqueue(self,item):
        self.items.insert(0,item)
    def dequeue(self):
        return self.items.pop()
    def size(self):
        return len(self.items)
    
class Stack():
    def __init__(self,initial):
        self.items = [initial]
    def isEmpty(self):
        items = []
    def push(self,item):
        self.items.append(item)
    def pop(self):
        return self.items.pop()
    def size(self):
        return len(self.items)

class PriorityQueue():
    def __init__(self,value,priorty):
        self.items = {priorty:[value]}
        
    def is_empty(self):
        if len(self.items) == 0:
          return True
        return False
    
    def enqueue(self,value,priorty):
        if priorty in(self.items.keys()):
            self.items[priorty].append(value)
        else:
            self.items[priorty] = [value]

    def dequeue(self): 
        if self.is_empty():
            return -1
        index = list(self.items.keys())
        index.sort()
        if len(self.items[index[0]]) != 0:
            z = self.items[index[0]].pop(0)
            if len(self.items[index[0]]) == 0:
                del self.items[index[0]]
                index.pop(0)
            return z

    
class State():
    right = {0,1,3,4,6,7}
    left = {1,2,4,5,7,8}
    up = {3,4,5,6,7,8}
    down = {0,1,2,3,4,5}
    
    def __init__(self, board_config, parent, move, depth):
        self.board_config = board_config #board configuration of the current state in a string
        self.board_config_list = list(map(int,board_config.split(','))) #board configuration of the current state in a list
        self.i = self.board_config_list.index(0) #index of empty space in board (index of 0 in this case)
        self.parent = parent #parent state (node) of the present state
        self.move = move #the move (Up,Down,Left,Right) made in parent state that results in the present state 
        self.depth = depth #depth of the node in the search tree
        
    def get_children(self):
        """returns the list of all possible states reachable from the current state,
        each child in the list is a State object"""
        children = []
        if self.i in State.up:
            new_board_config = self.board_config_list[:]
            new_board_config[self.i],new_board_config[self.i-3] = new_board_config[self.i-3],new_board_config[self.i]
            children.append(State(','.join(map(str,new_board_config)),self.board_config,'Up',self.depth+1))
        if self.i in State.down:
            new_board_config = self.board_config_list[:]
            new_board_config[self.i],new_board_config[self.i+3] = new_board_config[self.i+3],new_board_config[self.i]
            children.append(State(','.join(map(str,new_board_config)),self.board_config,'Down',self.depth+1))
        if self.i in State.left:
            new_board_config = self.board_config_list[:]
            new_board_config[self.i],new_board_config[self.i-1] = new_board_config[self.i-1],new_board_config[self.i]
            children.append(State(','.join(map(str,new_board_config)),self.board_config,'Left',self.depth+1))
        if self.i in State.right:
            new_board_config = self.board_config_list[:]
            new_board_config[self.i],new_board_config[self.i+1] = new_board_config[self.i+1],new_board_config[self.i]
            children.append(State(','.join(map(str,new_board_config)),self.board_config,'Right',self.depth+1))
        return children
    def __str__(self):
        return self.board_config
   
def bfs(initial,goal):
    """takes the initial and goal states as input;returns the sequence of 
    moves required to get from the start to goal 
    and also the total number of moves"""
    
    frontier=Queue(initial)
    frontier_set = set()
    explored_set=set()
    family_tree={}
    path=[]
    while not frontier.isEmpty():
        state = frontier.dequeue()
        explored_set.add(state.board_config)
        family_tree[state.board_config]=state
        
        if goal==state.board_config:
            current = state
            while current.parent != None:
                path.insert(0,current.move)
                current = family_tree[current.parent]
            return path, len(path)

        for child in state.get_children():
            if child.board_config not in frontier_set and child.board_config not in explored_set:
                frontier.enqueue(child)
                frontier_set.add(child.board_config)
    
    return "Failure"
    
def dfs(initial,goal):
    """takes the initial and goal states as input;returns the sequence of 
    moves required to get from the start to goal 
    and also the total number of moves"""
    
    frontier=Stack(initial)
    frontier_set = set()
    explored_set=set()
    family_tree={}
    path=[]
    while not frontier.isEmpty():
        state = frontier.pop()
        explored_set.add(state.board_config)
        family_tree[state.board_config]=state
        
        if goal==state.board_config:
            current = state
            while current.parent != None:
                path.insert(0,current.move)
                current = family_tree[current.parent]
            return path, len(path)

        for child in state.get_children():
            if child.board_config not in frontier_set and child.board_config not in explored_set:
                frontier.push(child)
                frontier_set.add(child.board_config)
    
    return "Failure"

def generate_dictionary(): 
    x = (0,1,2)
    n = 0
    dictionary = {}
    for a in x:
        for b in x:
            dictionary[n]=[a,b]
            n=n+1
    return dictionary

def hurestic(board_config):
    dictionary = generate_dictionary()
    board_config_list = board_config.split(",")
    distance = 0
    for j in board_config_list:
        i = int(j)
        if i != 0:
            z = board_config_list.index(str(i))
            vertical_distance = abs(dictionary[i][1]-z%3)
            horizontal_distance = abs(dictionary[i][0]-z//3)
            distance = distance + vertical_distance + horizontal_distance
    return distance

def ast(initial,goal):
    """takes the initial and goal states as input;returns the sequence of 
    moves required to get from the start to goal 
    and also the total number of moves"""
    frontier=PriorityQueue(initial,"0")
    frontier_set = set()
    explored_set=set()
    family_tree={}
    path=[]
    while not frontier.is_empty():
        state = frontier.dequeue()
        explored_set.add(state.board_config)
        family_tree[state.board_config]=state
        
        if goal==state.board_config:
            current = state
            while current.parent != None:
                path.insert(0,current.move)
                current = family_tree[current.parent]
            return path, len(path)

        for child in state.get_children():
            if child.board_config not in frontier_set and child.board_config not in explored_set:
                frontier.enqueue(child, hurestic(child.board_config))
                frontier_set.add(child.board_config)

    return 'Failure'


In [8]:
if __name__ == "__main__":
    start = '5,3,0,7,1,2,8,4,6'
    goal = '0,1,2,3,4,5,6,7,8'
    initial_state = State(start,None,None,0)
    
    import time
    
    start = time.time()
    print(bfs(initial_state,goal)) #should return (['Down', 'Down', 'Left', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Up', 'Left', 'Up'], 24)
    end = time.time()
    print("bfs:",end-start) #print the time taken for execution

    start = time.time()
    print(dfs(initial_state,goal))  #should return (['Left', 'Left', 'Down', 'Right', 'Right', ... , 'Down', 'Left', 'Up', 'Up'], 66482)
    end = time.time()
    print("dfs:",end-start) #print the time taken for execution
    
    start = time.time()
    print(ast(initial_state,goal)) #(['Down', 'Left', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Right', 'Down', 'Left', 'Left', 'Up', 'Up', 'Right', 'Down', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Up', 'Right', 'Down', 'Down', 'Left', 'Left', 'Up', 'Right', 'Down', 'Right', 'Up', 'Left', 'Left', 'Up'], 36)
    end = time.time()
    print("ast:",end-start) #print the time taken for execution

(['Down', 'Down', 'Left', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Up', 'Left', 'Up'], 24)
bfs: 3.4212050437927246
(['Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Up', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Up', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Up', 'Left', 'Left', 'Down', 'Right', 'Right', 'Down'